In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cx_Oracle

In [2]:
conn = cx_Oracle.connect("hr3", "hoseo","localhost:1521/xe") # 사용자이름, 비번, 서버위치
cursor = conn.cursor()  # cursor : 명령 처리기 

In [3]:
# create table   ## date 설정할때 이름 date로 할수 없음. 명칭 잘 확인 할것!
cursor.execute("""create table navernews(no number primary key, title varchar2(1000 char), link varchar2(1000 char), regdate varchar(1000 char), content varchar2(4000 char))""")   # oracle create table  

In [62]:
# 데이터베이스 컬럼 타입 변경
# cursor.execute("""alter table navernews modify no number""")

In [ ]:
# 테이블 삭제
# cursor.execute("""drop table navernews""")

In [63]:
# cursor.execute("create table contact(c_no number primary key,c_name varchar2(50 char) not null,c_phone varchar2(20 char) not null,c_email varchar2(50 char) null,c_regdate date default (sysdate))")

In [121]:
news = pd.read_csv('data-files1/news_크롤링.csv', encoding='utf-8')
news.columns
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  27 non-null     int64 
 1   제목          27 non-null     object
 2   링크          27 non-null     object
 3   날짜          27 non-null     object
 4   내용          27 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.2+ KB


In [122]:
news=news.rename(columns={"Unnamed: 0":"No", "제목":"Title", "링크":"Link" ,"날짜":"RegDate", "내용":"Content"})

In [123]:
news["No"].unique(), len(news["No"])

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26], dtype=int64),
 27)

In [124]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   No       27 non-null     int64 
 1   Title    27 non-null     object
 2   Link     27 non-null     object
 3   RegDate  27 non-null     object
 4   Content  27 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.2+ KB


In [125]:
news['No'] = news['No'].astype('int')
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   No       27 non-null     int32 
 1   Title    27 non-null     object
 2   Link     27 non-null     object
 3   RegDate  27 non-null     object
 4   Content  27 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.1+ KB


In [197]:
# rows = [tuple(x) for x in news.to_records(index=False)]
### content 내용이 길어 최대 1600 char 까지만 가능함.)
rows = [(int(x[0]), x[1], x[2], x[3], x[4][:1600]) for x in news.to_records(index=False)]
# rows = [(str(x[0]), x[1], x[2], x[3], x[4][:3000]) for x in news.to_records(index=False)]
type(rows)
rows[:1]
# news.head()
# news_list=list(news.values)

[(0,
  '황희찬, 한국 A매치 역대 최단시간 16초 만에 득점 신기록',
  'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012021604&isYeonhapFlash=Y&rc=N',
  '2020.11.17. 오후 10:31',
  " 카타르와 평가전서 황의조 어시스트로 '선제 벼락골' 작성   황희찬 선제골(서울=연합뉴스) 17일 오후 오스트리아 마리아 엔처스도르프의 BSFZ 아레나에서 열린 한국과 카타르의 축구대표팀 평가전. 황희찬이 선제골을 터트린 뒤 기뻐하고 있다. 2020.11.17 [대한축구협회 제공, 재판매 및 DB 금지] photo@yna.co.kr   (서울=연합뉴스) 안홍석 기자 = 한국 축구대표팀의 '황소' 황희찬(24·라이프치히)이 A매치 최단 시간 골 기록을 썼다.   황희찬은 17일(한국시간) 오스트리아 마리아 엔처스도르프의 BSFZ 아레나에서 열린 카타르와의 평가전에서 전반 16초 만에 선제골을 터뜨렸다.   이는 대한축구협회(KFA)가 확보한 기록상 남자 대표팀의 A매치 역대 최단 시간 득점이다.   이전까지 이 부문 1위는 박성화 전 경남FC 감독이 1979년 9월 14일 바레인과의 박대통령컵 국제대회 경기에서 20초 만에 넣은 득점이다.   황의조(보르도)가 적극적으로 상대 수비진을 압박해 골 지역 왼쪽에서 공을 탈취, 문전으로 패스했고, 황희찬이 가볍게 슈팅해 새 기록을 썼다.   황희찬은 이 골로 A매치 34경기 5골을 기록했다.   ahs@yna.co.kr   ▶코로나19 속보는 네이버 연합뉴스[구독 클릭]▶[팩트체크]조두순 출소후 법으로 음주 금지 가능?▶제보하기   <저작권자(c) 연합뉴스(https://www.yna.co.kr/), 무단 전재-재배포 금지> 기사제공 연합뉴스  ")]

In [198]:
sql = """INSERT INTO navernews(no, title, link, regdate, content) 
         VALUES (:no, :title, :link, :regdate, :content)"""
cursor.executemany(sql, rows)
conn.commit()
 
# cursor.execute('SELECT COUNT(*) FROM ITEM_CODE')
# cnt = cursor.fetchone()
 
# print('Insert 건수: ', cnt)

In [4]:
# cursor.execute("""drop table navernews""")
# conn.commit()

In [205]:
cursor.close()
conn.close()

In [183]:
rows[25]

(25,
 "'3명 사망' 인천 공장 화재…화학물질 폭발로 인명피해 커져",
 'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012027418&isYeonhapFlash=Y&rc=N',
 '2020.11.19. 오후 7:29',
 '     소방당국 남성 직원들이 여성 직원들 먼저 대피시켜   화재로 근로자 3명 사망한 인천 화장품 제조공장(인천=연합뉴스) 윤태현 기자 = 19일 오후 4시 12분께 인천시 남동구 고잔동 한 화장품 제조업체 공장에서 불이 나 공장 관계자 3명이 숨지고 6명이 다쳐 인근 병원으로 옮겨지거나 현장에서 응급 처치를 받았다. 사진은 화재 현장 모습. 2020.11.19 tomatoyoon@yna.co.kr   (인천=연합뉴스) 손현규 윤태현 기자 = 근로자 3명이 숨지고 6명이 다친 인천 화장품 제조공장 화재는 화학물질을 이용한 작업 중 폭발과 함께 일어난 것으로 추정됐다.   비교적 어둡지 않은 오후에 작업 도중 발생한 화재이지만 갑작스러운 화학물질 폭발로 근로자들이 미처 대피하지 못한 탓에 인명피해가 커졌다.      이재선 공단소방서 현장대응단장은 19일 화재가 발생한 인천시 남동구 고잔동 한 화장품 제조업체 공장 인근에서 브리핑을 열고 공장에서 아염소산나트륨을 사용하고 있다고 진술을 받았다고 말했다.   그는 이어 아염소산나트륨은 화염이 발생했을 때 (반응하면) 폭발할 가능성이 있는 물질이라고 덧붙였다.   소방당국은 공장 내에서 도금 작업을 하던 중 펑하는 소리와 함께 폭발이 일어난 뒤 화재가 발생한 것으로 보고 있다.   화재로 근로자 3명 사망한 인천 화장품 제조공장(인천=연합뉴스) 윤태현 기자 = 19일 오후 4시 12분께 인천시 남동구 고잔동 한 화장품 제조업체 공장에서 불이 나 공장 관계자 3명이 숨지고 6명이 다쳐 인근 병원으로 옮겨지거나 현장에서 응급 처치를 받았다. 사진은 화재 현장 모습. 2020.11.19 tomatoyoon@yna.

In [ ]:
# db.execute(sql_insert, BRAND=p_tag.encode('utf8').decode('utf8'), NAME=name_tag.encode('utf8').decode('utf8'), PRICE=span_tag.encode('utf8').decode('utf8'), SRC=img_tag.get('data-desktop-src').encode('utf8').decode('utf8'))